$p_{\theta}(s_{t+1}|s_t, a_t) = \{_{pdf(x_{t+1}, \; (x_t+a_t^0), \; \theta_2) \; \times \; pdf(y_{t+1}, \; (y_t+a_t^1), \; \theta_2) \; if \; x_t >= \theta_0}^{pdf(x_{t+1}, \; (x_t+a_t^0), \; \theta_1) \; \times \; pdf(y_{t+1}, \; (y_t+a_t^1), \; \theta_1) \; if \; x_t < \theta_0}$

In [155]:
import numpy as np
import random

theta = [5, (2,4)]

def transiction(s, a, p=2):
    return tuple(np.array(s) + np.array(a) + np.random.normal(0, p, 2))

def pickModel(s, theta):
    psi, sigma = theta
    x,_ = s
    return sigma[int(x >= psi)]

def generate(s, n, theta):   
    for _ in range(n):
        a = random.choice([(1,0), (-1,0), (0,1), (0,-1)])
        yield s, a
        s = transiction(s, a, pickModel(s, theta))

s_0 = (1,1)
O = [(s, a) for (s, a) in generate(s_0, 100, theta)]
O

[((1, 1), (1, 0)),
 ((1.5188314264394287, 0.5596376883141133), (-1, 0)),
 ((-1.1949229485312567, -4.649901097556078), (-1, 0)),
 ((-4.861304473651968, -3.363642314607672), (-1, 0)),
 ((-7.07592364763015, -1.4702861987006082), (0, -1)),
 ((-3.294625000798343, -2.2159424969472026), (-1, 0)),
 ((-7.713388056230321, -5.725450298819245), (0, 1)),
 ((-9.04880939500287, -5.583638139331882), (0, -1)),
 ((-9.090376190328207, -4.170825980559461), (1, 0)),
 ((-6.259421081016324, -4.3459095778345835), (-1, 0)),
 ((-4.680824013063962, -5.999302007259261), (-1, 0)),
 ((-5.769969996958715, -2.558314162401247), (-1, 0)),
 ((-6.0757889674823335, -0.23656896256920756), (-1, 0)),
 ((-10.725693501195899, -3.6768229052759724), (-1, 0)),
 ((-11.144554963579989, -3.804437264965139), (0, 1)),
 ((-9.52052043806917, -4.395347636350924), (1, 0)),
 ((-9.925902602777567, -4.513901501613909), (0, 1)),
 ((-13.346198509471916, -2.4339635289078094), (1, 0)),
 ((-8.417505948400404, -3.4918558379070834), (0, -1)),
 ((-9

$ \theta = \{\psi, \sigma \} $

$\psi$: Parametros para definição do modelo que será utilizado em cada estado<br>
$\sigma$: Parametros para cada Modelo $\{\sigma_1, \sigma_2, ...\}$ <br>

$p_{\theta}(s_{t+1}|s_t, a_t) = \mu(s_{t+1}, s_t, a_t, \sigma_1) \; \times \; \beta(s_t) + \mu(s_{t+1}, s_t, a_t, \sigma_2) \; \times \; \beta(s_t) + ...$



$p_{\theta}(s_{t+1}|s_t, a_t) = \sum_{i=1}^k \mu(s_{t+1}, s_t, a_t, \sigma_i) \beta(s_t)_i \;\;\;\;\; for \;\; k = |\sigma_{\theta}|$  


$\mu: S \times S \times A \times \sigma_{\theta} \mapsto [0,1]; \;\; \mu(s', s, a, \sigma) = pdf(s'_x, (s_x+a_x), \sigma) \; \times \; pdf(s'_y, (s_y+a_y), \sigma)$

##### 2 modelos: Sigmoid

$ \beta: S \mapsto [0,1]; \;\; \beta(s; \psi_{\theta})_i = (i-1) + \frac{1}{1+e^{\psi_{s}}}(-1)^{i-1} $;  <br>
$ for \;\; i = \{1,2\} $

##### N modelos: Softmax

$ \beta: S \mapsto [0,1]; \;\; \beta(s; \psi_{\theta})_i = argmax[\frac{e^{\psi_{s,i}}}{\sum_{j=1}^k e^{\psi_{s,j}}}] $;   <br>
$ for \;\; k = |\sigma_{\theta}| \;\; ; \;\; i=1,...,k $

$\cal{L}[\xi | \theta] = \sum_{t=0}^{\mathrm{T}-1} log \; p_{\theta}(s_{t+1}|s_t, a_t)$

$\cal{L}[\xi | \theta] = \sum_{t=0}^{\mathrm{T}-1} log \; p_{\theta}(s_{t+1}|s_t, a_t) - \gamma$

$ \gamma = \sum_t \sum_{s\;vizinho\;s_t} (1-\beta(s)) * \beta(s_t) + \beta(s) * (1-\beta(s_t)) $

$ \gamma = \sum_t \sum_{s\;vizinho\;s_t} KL(p(s), p(s_t))$ <br>
KL = Kullback-Leibler 

$ \gamma = \sum_t \sum_{s} KL(p(s), p(s_t)) * e^{-|s_t - s|} $

$ KL(P||Q) = \sum_x P(x) log (\frac{P(x)}{Q(x)}) $

In [ ]:
from scipy.stats import norm

def ll(O, theta, c=10):
    psi, *sig = theta

    beta = lambda x,y, theta: c * (theta-x)
    p = lambda x_,y_, x,y, ax,ay: ( 
        norm.pdf(x_, (x + ax), sig[1])  
        * norm.pdf(y_, (y + ay), sig[1]) 
        * (1/(1 + np.e **beta(x,y, psi)))
        +
        norm.pdf(x_, (x + ax), sig[0]) 
        * norm.pdf(y_, (y + ay), sig[0])
        * (1 - (1/(1 + np.e **beta(x,y, psi))))
    )

    S,A = list(zip(*O))
    
    ps = lambda x_, y_: np.sum([p(x_,y_, x,y, ax,ay) for (x,y) in S for (ax,ay) in A])
    d = lambda p1, p2: (1-p1)*p2 + p1*(1-p2)
    gamma = np.sum([d(ps(x,y), ps(xt,yt)) for t,((xt,yt),_) in enumerate(O) for (x,y) in S if (abs(x-xt)<1.5 and abs(y-yt)<1.5)])
    return np.sum([np.log(p(*st, *O[t-1][0], *O[t-1][1])) for t,(st,_) in enumerate(O) if 0 < t]) - gamma

ll(O, (5, 2, 4), c=10)

In [152]:
from scipy.stats import norm

def ll(O, theta, c=10):
    psi, *sig = theta

    beta = lambda x,y, theta: c * (theta-x)
    p = lambda x_,y_, x,y, ax,ay: ( 
        norm.pdf(x_, (x + ax), sig[1])  
        * norm.pdf(y_, (y + ay), sig[1]) 
        * (1/(1 + np.e **beta(x,y, psi)))
        +
        norm.pdf(x_, (x + ax), sig[0]) 
        * norm.pdf(y_, (y + ay), sig[0])
        * (1 - (1/(1 + np.e **beta(x,y, psi))))
    )

    S,A = list(zip(*O))
    
    ps = lambda x_, y_: np.sum([p(x_,y_, x,y, ax,ay) for (x,y) in S for (ax,ay) in A])
    d = lambda p1, p2: (1-p1)*p2 + p1*(1-p2)
    gamma = np.sum([d(ps(x,y), ps(xt,yt)) for t,((xt,yt),_) in enumerate(O) for (x,y) in S if (abs(x-xt)<1.5 and abs(y-yt)<1.5)])
    return np.sum([np.log(p(*st, *O[t-1][0], *O[t-1][1])) for t,(st,_) in enumerate(O) if 0 < t]) - gamma

ll(O, (5, 2, 4), c=10)

-41.989061021388764

In [156]:
from scipy.stats import norm

def ll(O, theta, c=10):
    psi, *sig = theta

    beta = lambda x,y, theta: c * (theta-x)
    
    _p = {}
    def p(x_,y_, x,y, ax,ay):
        if (x_,y_, x,y, ax,ay) not in _p:
            _p[(x_,y_, x,y, ax,ay)] = ( 
                norm.pdf(x_, (x + ax), sig[1])  
                * norm.pdf(y_, (y + ay), sig[1]) 
                * (1/(1 + np.e **beta(x,y, psi)))
                +
                norm.pdf(x_, (x + ax), sig[0]) 
                * norm.pdf(y_, (y + ay), sig[0])
                * (1 - (1/(1 + np.e **beta(x,y, psi))))
            )
        return _p[(x_,y_, x,y, ax,ay)]

    S,A = list(zip(*O))
    
    _ps = {}
    def ps(x_,y_):
        if (x_,y_) not in _ps:
            _ps[(x_,y_)] = np.sum([p(x_,y_, x,y, ax,ay) for (x,y) in S for (ax,ay) in A])
        return _ps[(x_,y_)]
                   
    d = lambda p1, p2: (1-p1)*p2 + p1*(1-p2)
    gamma = np.sum([d(ps(x,y), ps(xt,yt)) for t,((xt,yt),_) in enumerate(O) for (x,y) in S if (abs(x-xt)<1.5 and abs(y-yt)<1.5)])
    return np.sum([np.log(p(*st, *O[t-1][0], *O[t-1][1])) for t,(st,_) in enumerate(O) if 0 < t]) - gamma

ll(O, (5, 2, 4), c=10)

613915.6865961527

In [154]:
# best_theta = 0
# best_ll = -np.inf

# for psi in range(10):
#     for p1 in range(10):
#         for p2 in range(10):
#             theta = (psi+1, p1+1, p2+1)
#             new_ll = ll(O, theta)
#             if new_ll > best_ll:
#                 best_ll = new_ll
#                 best_theta = theta

# print(best_theta, best_ll)

(3, 1, 4) -19.74068303836114


In [170]:
import torch
from torch.autograd import Variable
from torch.distributions import normal

s = Variable(torch.from_numpy(np.array(O)[:-1,0])).type(torch.DoubleTensor)
a = Variable(torch.from_numpy(np.array(O)[:-1,1])).type(torch.IntTensor)
s_ = Variable(torch.from_numpy(np.array(O)[1:,0])).type(torch.DoubleTensor)

us = torch.unique(s, dim=0)
ua = torch.unique(a, dim=0)

p = torch.nn.parameter.Parameter(torch.Tensor(3).uniform_(1, 10))

c=10
beta = c * (p[0] - s[:,0]) 

pTheta = (
        torch.exp(normal.Normal(s[:,0] + a[:,0], p[2]).log_prob(s_[:,0]))
        * torch.exp(normal.Normal(s[:,1] + a[:,1], p[2]).log_prob(s_[:,1]))
        * torch.sigmoid(-beta)
        + 
        torch.exp(normal.Normal(s[:,0] + a[:,0], p[1]).log_prob(s_[:,0]))
        * torch.exp(normal.Normal(s[:,1] + a[:,1], p[1]).log_prob(s_[:,1]))
        * (1 - torch.sigmoid(-beta))
    )


In [225]:
import torch
from torch.autograd import Variable
from torch.distributions import normal

def estimate(beta=1.5, init_p=None):
    s = Variable(torch.from_numpy(np.array(O)[:-1,0])).type(torch.DoubleTensor)
    a = Variable(torch.from_numpy(np.array(O)[:-1,1])).type(torch.IntTensor)
    s_ = Variable(torch.from_numpy(np.array(O)[1:,0])).type(torch.DoubleTensor)

    us = torch.unique(s)

    if init_p is not None:
        p = torch.nn.parameter.Parameter(torch.tensor(init_p))
    else:
        p = torch.nn.parameter.Parameter(torch.Tensor(3).uniform_(1, 10))


    pTheta = (
        torch.exp(normal.Normal(s[:,0] + a[:,0], p[2]).log_prob(s_[:,0]))
        * torch.exp(normal.Normal(s[:,1] + a[:,1], p[2]).log_prob(s_[:,1]))
        * torch.sigmoid(-beta * (p[0]-s[:,0]))
        + 
        torch.exp(normal.Normal(s[:,0] + a[:,0], p[1]).log_prob(s_[:,0]))
        * torch.exp(normal.Normal(s[:,1] + a[:,1], p[1]).log_prob(s_[:,1]))
        * (1 - torch.sigmoid(-beta * (p[0]-s[:,0])))
    )

    probs = torch.sum(pTheta[s])
    gamma = lambda: (
        torch.sum()
    )

    loglike = lambda: -torch.sum(torch.log(pTheta)) + gamma

    # optim = torch.optim.SGD([p], lr=1e-2, momentum=0.9)
    optim = torch.optim.SGD([p], lr=1e-2)

    for i in range(1000):
        # print(i, p)
        ll = loglike()
        optim.zero_grad()
        ll.backward()
        optim.step() 

        with torch.no_grad():
            p[:] = p.clamp(min=1e-1)

    return p


estimate(2)

Parameter containing:
tensor([5.5514, 1.9257, 4.2273], requires_grad=True)

In [233]:
start = [1., 1., 1.]

for i in range(10):
    print(i*.5, estimate(i*.5, start))

0.0 Parameter containing:
tensor([1.0000, 2.4884, 2.4884], requires_grad=True)
0.5 Parameter containing:
tensor([8.2470, 1.9428, 4.9115], requires_grad=True)
1.0 Parameter containing:
tensor([6.5869, 1.9313, 4.5599], requires_grad=True)
1.5 Parameter containing:
tensor([5.8309, 1.9270, 4.3175], requires_grad=True)
2.0 Parameter containing:
tensor([5.5514, 1.9257, 4.2273], requires_grad=True)
2.5 Parameter containing:
tensor([5.4274, 1.9251, 4.1834], requires_grad=True)
3.0 Parameter containing:
tensor([5.3500, 1.9250, 4.1500], requires_grad=True)
3.5 Parameter containing:
tensor([5.2903, 1.9254, 4.1201], requires_grad=True)
4.0 Parameter containing:
tensor([5.2436, 1.9259, 4.0949], requires_grad=True)
4.5 Parameter containing:
tensor([5.2107, 1.9265, 4.0764], requires_grad=True)


In [253]:
for i in range(10):
    n = [[round(n, 2) for n in estimate(i*.5).data.tolist()] for _ in range(5)]
    print(i*.5, *n)

0.0 [2.37, 3.19, 1.62] [4.57, 1.62, 3.19] [5.14, 1.62, 3.19] [6.27, 1.62, 3.19] [5.37, 1.62, 3.19]
0.5 [8.25, 1.94, 4.91] [8.28, 1.94, 4.92] [8.22, 1.94, 4.9] [8.33, 1.94, 4.94] [8.27, 1.94, 4.92]
1.0 [6.59, 1.93, 4.56] [6.59, 1.93, 4.56] [6.59, 1.93, 4.56] [6.59, 1.93, 4.56] [6.59, 1.93, 4.56]
1.5 [5.83, 1.93, 4.32] [5.83, 1.93, 4.32] [5.83, 1.93, 4.32] [5.83, 1.93, 4.32] [5.83, 1.93, 4.32]
2.0 [5.55, 1.93, 4.23] [5.55, 1.93, 4.23] [5.55, 1.93, 4.23] [5.55, 1.93, 4.23] [5.55, 1.93, 4.23]
2.5 [5.43, 1.93, 4.18] [5.43, 1.93, 4.18] [5.43, 1.93, 4.18] [5.43, 1.93, 4.18] [5.43, 1.93, 4.18]
3.0 [5.35, 1.93, 4.15] [5.35, 1.93, 4.15] [5.35, 1.93, 4.15] [5.35, 1.93, 4.15] [5.35, 1.93, 4.15]
3.5 [8.47, 2.19, 4.66] [5.29, 1.93, 4.12] [5.29, 1.93, 4.12] [5.29, 1.93, 4.12] [8.47, 2.19, 4.66]
4.0 [5.24, 1.93, 4.09] [5.24, 1.93, 4.09] [5.24, 1.93, 4.09] [5.24, 1.93, 4.09] [5.24, 1.93, 4.09]
4.5 [8.52, 2.19, 4.64] [8.52, 2.19, 4.64] [8.52, 2.19, 4.64] [5.21, 1.93, 4.08] [5.21, 1.93, 4.08]
